In [ ]:
# First pip install smolagents
!pip install smolagents -q

In [ ]:
# Install the dependencies and login to HF account to access the Inference API
from huggingface_hub import notebook_login

notebook_login()

# THE TOOL DECORATORS
Generating a tool that retrieves the highest-rated restaurants in Berlin.

We will create an example of how we can use @tool decorator to find out best restaurant in Berlin.

In [ ]:
from smolagents import CodeAgent, InferenceClientModel, tool

# Let's pretend we have a function that fetches the highest-rated restaurant services.


@tool
def restaurant_service_tool(query: str) -> str:
  """
  This tool returns the highest rated restaurant service in Berlin.

  Args:
      query: A search term for finding restaurant services
  """
  # Example list of restaurants and their ratings
  restaurants = {
      "Restaurant-1":4.9,
      "Resturant-2":4.7,
      "McDonalds":4.1,
      "Starbucks":3.7,
      "Restaurant-3": 4.8
  }
  # Find the highest rated rest. (simulating search query filtering)
  best_restaurant = max(restaurants, key=restaurants.get)

  return best_restaurant

agent = CodeAgent(
    tools=[restaurant_service_tool],
    model=InferenceClientModel()
)

# Run the Agent to find the best restaurant.
result = agent.run(
    "Can you give me the name of the highest rated restaurant service in Berlin City?"
)

print(result)

# Defining a Tool as Python Class

Generating a tool to generate ideas about the phd ideas

Alfred needs some creative ideas to make it special.

To do this, he can use an agent that generates phd ideas  based on a given category.

In [ ]:
from smolagents import Tool, CodeAgent, InferenceClientModel

class PhdIdeasTopicTool(Tool):
  name = "phd_ideas_generator"
  description = """
  This tool suggests creative phd ideas based on category.
  It returns a unique phd idea."""

  inputs = {
      "category":{
          "type":"string",
          "description":"The type of phd Idea (e.g, 'computational Linguistics', 'Natural Language Processing', 'Computer-science related (more technical)')"
      }
  }

  output_type = "string"

  def forward(self, category: str):
    themes = {
        "computational linguistics:This category has more linguistics elements with enough computational side",
        "Natural Language Processing: This is related to modern NLP tasks and ML algorithms",
        "Computer Science: This is more in the technical side rather than relevant to linguistics aspects of NLP."
    }

    return themes.get(category.lower(), "Themed phd idea not found. Try 'computational linguistics', 'Natural Langauge Processing', or 'Computer Science'")

# Instantiate the tool
phd_idea_tool = PhdIdeasTopicTool()
agent = CodeAgent(tools=[phd_idea_tool], model=InferenceClientModel())
# run the agent to generate a phd idea with a specific category.
result = agent.run(
    "What would be a good phd idea for a computational linguistics theme?"
)

print(result)

# Sharing a tool the hub

Sharing your custom tool with the community is easy! Simply upload it to your Hugging Face account using the push_to_hub() method.

For instance, Alfred can share his catering_service_tool to help others find the best catering services in Gotham. Here's how to do it:

In [ ]:
phd_idea_tool.push_to_hub("CanerCoban/phd_ideas_tool", token="MYHFtOKEN")

# Importing a Tool from the Hub

You can easily import tools created by other users using the load_tool() function.

Instead of building a tool form scratch, we can use a predefined one from the community

In [ ]:
from smolagents import load_tool, CodeAgent, InferenceClientModel

image_generation_tool = load_tool(
    "m-ric/text-to-image",
    trust_remote_code=True
)

agent = CodeAgent(
    tools=[image_generation_tool],
    model=InferenceClientModel()
)

agent.run("Generate an image of graduate student looking for phd jobs.")

# Importing a HuggingFace as a tool
We can import a HF Space as a tool using Tool.from_space().


Opens up possibilities for integrating with thousands of spaces from the community for tasks from image generation to data analysis.

In [ ]:
# The tool will connect with the spaces Gradio backend using the gradio_client
# we need to make sure that gradio is installed via pip.
!pip install gradio_client

In [ ]:
from smolagents import CodeAgent, InferenceClientModel, Tool

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)

model = InferenceClientModel("Qwen/Qwen2.5-Coder-32B-Instruct")

agent = CodeAgent(tools=[image_generation_tool],model=model)

agent.run("Improve this prompt, then generate an image of it.",
          additional_args={'user_prompt': 'A graduate student looking for phd positions to apply.'}
          )

In [ ]:
from PIL import Image as PILImage
import matplotlib.pyplot as plt

image_path = '/tmp/gradio/ee85723f43a3b52e4b0f8656204aefd908298d0577c71647607732c1cf073314/image.webp'

img = PILImage.open(image_path)
img

# Importing a Langchain tool

These ttols need a (https://serpapi.com/) Google Search API key.

You can easily load LangChain tools using the Tool.from_langchain() method.

We need to tap into LangChain tools to find top-tier entertainment tools.


By using Tool.from_langchain(), we add search functionalities to our smolagents, enable us to discover exclusive party ideas and services with just a few commands.

Here is how we do it:

In [ ]:
!pip install langchain-community google-search-results

In [ ]:
from google.colab import userdata

import os

os.environ["SERPAPI_API_KEY"] = userdata.get("SEARCH_API")

In [ ]:
from langchain.agents import load_tools
from smolagents import CodeAgent, InferenceClientModel, Tool

search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

agent = CodeAgent(tools=[search_tool], model=model)

agent.run("Search for phd ideas for a computational linguist graduate student")


# Importing a tool collection from any MCP server

 smolagents also allows importing tools from the hundreds of MCP servers available on glama.ai or smithery.ai.

In [ ]:
# Install mcp client
# We first need to install the mcp integration for smolagents
!pip install "smolagents[mcp]"

In [ ]:
# The MCP server tools can be loaded in a ToolCollecttion object as follow:
import os
from smolagents import ToolCollection, CodeAgent
from mcp import StdioServerParameters
from smolagents import InferenceClientModel


model = InferenceClientModel("Qwen/Qwen2.5-Coder-32B-Instruct")


server_parameters = StdioServerParameters(
    command="uvx",
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], model=model, add_base_tools=True)
    agent.run("Please find a remedy for hangover.")